## Poker excise 

set(): dosen't keep duplicate copies and is unodered
tuple is an ummutable

In [ ]:
## Docstrings give you an easy way of associating documentation with a function.

In [51]:
def poker(hands):
    """Retrun a list of winning hands: poker([hand, ...])->[handd, ...]"""
    return allmax(hands, key=hand_rank)
    #return max(handle, key=hand_rank) # resolve tie
    # poker([hand, ...]) => hand

## Handle the tie, like ranks is same in all hands
def allmax(iterable, key=None):
    """Return a list of all items equal to max of the iterable"""
    result, maxval = [], None
    key = key or (lambda x: x)  ## key is optional function, Key is None use lambda
    for x in iterable:
        xval =  key(x)
        if not result or xval > maxval:
            result, maxval = [x], xval
        elif xval == maxval:
            result.append(x)
    return result

def hand_rank(hand):
    """Return a value indicating the ranking of a hand
       Ref to: https://en.wikipedia.org/wiki/List_of_poker_hands
    """
    ranks = card_ranks(set(hand)) # adding set(hand) to remove duplicate cards when do homework2
    if straight(ranks) and flush(hand): # straight flush
        return (8, max(ranks))
    elif kind(4, ranks):
        return (7, kind(4, ranks), kind(1, ranks)) # 4 of kind
    elif kind(3, ranks) and kind(2, ranks):  
        return (6, kind(3, ranks), kind(2, ranks)) # full house
    elif flush(hand):
        return (5, ranks)
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return (2, two_pair(ranks), ranks)
    elif kind(2, ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)
    

def card_ranks(hand):
    """Return a list of the rank, sorted by higher first.
    Ace can be rank to 14 or 1"""
    ranks = ['--23456789TJQKA'.index(r) for r, s in hand]
    ranks.sort(reverse=True)
    #L.sort(key=None, reverse=False)
    return [5,4,3,2,1] if (ranks == [14,5,4,3,2]) else ranks
    
def straight(ranks):
    """Return True if the ordered ranks from a 5card straigh"""
    return (max(ranks)-min(ranks) == 4) and len(set(ranks)) == 5   

def flush(hand):
    "Return True if all the cards have the same suit."
    suits = [s for r,s in hand]
    return len(set(suits)) == 1

def kind(n, ranks):
    """Return the first rank that this hand has exaclty n of
    Return None if there is no n of a kind in the hand"""
    for r in ranks:
        if ranks.count(r) == n: 
            return r
    return None

def two_pair(ranks):
    """If there are two pair, return tuple:(Highest, lowest).
    otherwise return None"""
    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks)))
    if pair and lowpair != pair:
        return (pair, lowpair)
    else:
        return None

In [102]:
import random

mydeck = [r+s for r in "2346789TJQKA" for s in "SHDC"]

In [103]:
def deal(numhands, n=5, deck=mydeck):
    "based on deal return deck"
    random.shuffle(deck)
    return [deck[n*i:n*(i+1)] for i in range(numhands)]

In [138]:
hand_names = ['Stright Flush:  0.001', 
             '4 kind:  0.019',
             'Full house:  0.146',
             'Flush:  0.198',
             'Strighat:  0.401',
             '3 kind:  2.228',
             '2 pair:  4.752',
             'Pair: 42.329',
             'High Card: 50.036']

def hand_percentages(n=700*1000):
    "Sample n random hands and print a table of percentages for each type of hand."
    counts = [0] * 9
    for i in range(n // 10):
        for hand in deal(10):
            ranking = hand_rank(hand)[0]
            counts[ranking] += 1
    for i in range(9):
        #print("%20s: %% %6.3f %%" % (hand_names[i], 100.*counts[i]/n))
        # {} and with : used instead of %
        print("{:>21}%, {:6.3f}%".format(hand_names[i], 100.*counts[i]/n))

In [139]:
hand_percentages()

Stright Flush:  0.001%, 49.771%
       4 kind:  0.019%, 41.857%
   Full house:  0.146%,  4.958%
        Flush:  0.198%,  2.256%
     Strighat:  0.401%,  0.266%
       3 kind:  2.228%,  0.684%
       2 pair:  4.752%,  0.177%
         Pair: 42.329%,  0.028%
    High Card: 50.036%,  0.002%


In [12]:
deal(10)

[['AH', 'KC', 'TD', '6D', 'KH'],
 ['4H', '9D', '6S', 'JC', '6H'],
 ['8H', '7H', '4C', '3C', '8S'],
 ['JH', '2C', '4S', '2S', '8D'],
 ['9S', '7C', '3S', '2D', '6C'],
 ['JS', 'TH', '8C', 'KS', '3H'],
 ['9H', 'AS', 'TC', 'QH', 'AD'],
 ['7D', 'TS', 'JD', '2H', '9C'],
 ['QS', '4D', '3D', 'KD', 'QD'],
 ['AC', '7S', 'QC']]

## Four dimensions of program:
>1. correctness
>2. efficiency
>3. features
>4. elegante

trader off uppper dimension by refactoring

*-Dry: Don't repeat myself-*

In [13]:
# Refactoring code: remove repeat code
def hand_rank(hand):
    """Return a value indicating how high the hand ranks."""
    # counts is the count of each work ranks list corresponding ranks
    # E.g. '7 T 7 9 7' => counts = (3, 1, 1), ranks = (7, 9, 10)
    groups = group(['--23456789TJQKA'.index(r) for r,s in hand])
    counts, ranks = unzip(groups)
    if ranks == (14, 5, 4, 3, 2):
        ranks =  (5, 4, 3, 2, 1)
    straight = len(ranks) ==  5 and max(ranks)-min(ranks) == 4
    flush =  len(set([s for r,s in hand])) == 1
    return (9 if (5,) == counts else
            8 if straight and flush else
            7 if (4, 1) == counts else
            6 if (3, 2) == counts else
            5 if flush else
            4 if straight else
            3 if (3, 1, 1) == counts else
            2 if (2, 2, 1) == counts else
            1 if (2, 1, 1, 1) ==  counts else
            0), ranks

def group(items):
    """Return a list of [(count, x)...], highest count first, then highest x first"""
    groups = [(items.counts(x), x) for x in set(items)]
    return sorted(groups, reverse=True)

def unzip(pairs): return zip(*pairs) ## *paris??

In [141]:
## solution 2

def hand_rank(hand):
    """Return a value indicating how high the hand ranks."""
    # counts is the count of each work ranks list corresponding ranks
    # E.g. '7 T 7 9 7' => counts = (3, 1, 1), ranks = (7, 9, 10)
    groups = group(['--23456789TJQKA'.index(r) for r,s in hand])
    counts, ranks = unzip(groups)
    if ranks == (14, 5, 4, 3, 2):
        ranks =  (5, 4, 3, 2, 1)
    straight = len(ranks) ==  5 and max(ranks)-min(ranks) == 4
    flush =  len(set([s for r,s in hand])) == 1
                 
    return max(counts_rankings[counts], 4*straight + 5*flush), ranks
                 
counts_rankings = {(5,):10, (4,1):7, (3,2):6, (3,1,1):3, (2,2,1):2, (2,1,1,1):1, (1,1,1,1,1):0}

In [151]:
# zip?

x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
zip(x, y, z)

Lesson Learn:
- Understand the question
- Define pieces of problem
- Reuse : remove repeated parts
- Test: Specify it

## Shuffle alorithm



In [159]:
def shuffe1(deck):
    N = len(deck)
    swapped = [False] * N
    while not all(swapped):
        i, j = random.randrange(N), random.randrange(N)
        swapped[i] = swapped[j] = True
        swap(deck, i, j)
        
        
def swap(deck, i, j):
    "Swap elements i and j of collection."
    print("swap", i, j)
    deck[i], deck[j] = deck[j], deck[i]

In [160]:
def shuffle(deck):
    "Knuth's Algorithm P."
    N = len(deck)
    for i in range(N-1): # not N as last one in swap itself
        swap(deck, i, random.randrange(i, N))

In [167]:
str.joint?

Object `str.joint` not found.


In [168]:
from collections import defaultdict

def test_shuffer(shuffler, deck='abcd', n=10000):
    counts =  defaultdict(int) # counts start from 0
    for _ in range(n):
        input = list(deck)
        shuffler(input)
        counts[''.join(input)] += 1
    e = n*1./factorial(len(deck)) ## expected count
    ok = all((0.9 <= counts[item]/e <= 1.1)
            for item in counts)
    name = shuffler.__name__
    print('{},{},{}'.format(name, deck, ('ok' if ok else '***BDA***')))
    print('    ')
    for item, count in sorted(counts.items()):
        print("%s:%4.1f" % (item, count*100./n))
    print()
    
def test_shufflers(shufflers=[shuffle, shuffle1], decks=['abc', 'ab']):
    for deck in decks:
        print()
        for f in shufflers:
            test_shuffer(f, deck)
            
def factorial(n): return 1 if (n <= 1) else n*factorial(n-1)

In [171]:
test_shuffer()

TypeError: test_shuffer() missing 1 required positional argument: 'shuffler'

In [ ]:
def shuffle2(deck):
    "A modification of my teacher's algorithm"
    N = len(deck)
    swapped = [False] * N
    while not all(swapped):
        i, j = randrange(N), randrange(N)
        swapped[i] = True
        swap(deck, i, j)
        
def shuffle3(deck):
    "An easier modification of my teacher's algorithm"
    N = len(deck)
    for i in range(N):
        swap(deck, i, randrange(N))

## List comprehensions:

[ref to](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)

In [1]:
## andy list comprehension

udacity_tas = ['Peter', 'Andy', 'Gundega', 'Job', 'Sean']

uppercase_tas = [name.upper() for name in udacity_tas]

print(uppercase_tas)

['PETER', 'ANDY', 'GUNDEGA', 'JOB', 'SEAN']


In [2]:
for name in uppercase_tas:
    print(name)

PETER
ANDY
GUNDEGA
JOB
SEAN


In [6]:
## use string.find()

#ta_300 = ['Gundega is the TA for cs373',
#         'Job is the TA for cs387'']

ta_data = [['Peter', 'USA', 'cs262'],
           ['Andy', 'USA', 'cs212'],
           ['Gundega', 'Lativa', 'cs232'],
           ['Sarah', 'England', 'cs373'],
           ['Job', 'USA', 'cs387'],
           ['Sean', 'USA', 'cs253']]

ta_facts = [name + ' lives in ' + country + ' and is the TA for ' + course for name, country, course in ta_data]

for row in ta_facts:
    print(row)

Peter lives in USA and is the TA for cs262
Andy lives in USA and is the TA for cs212
Gundega lives in Lativa and is the TA for cs232
Sarah lives in England and is the TA for cs373
Job lives in USA and is the TA for cs387
Sean lives in USA and is the TA for cs253


In [7]:
ta_300 = [name + ' is the TA for ' + course for name, country, course in ta_data if course.find('cs3') != -1]

for row in ta_300:
    print(row)

Sarah is the TA for cs373
Job is the TA for cs387


## Homework

In [10]:
## problem 1: 7-card stud

import itertools

def best_hand(hand):
    return max(itertools.combinations(hand, 5), key=hand_rank)

def test_best_hand():
    assert (sorted(best_hand("6C 7C 8C 9C TC 5C JS".split()))
            == ['6C', '7C', '8C', '9C', 'TC'])
    assert (sorted(best_hand("TD TC TH 7C 7D 8C 8S".split()))
            == ['8C', '8S', 'TC', 'TD', 'TH'])
    assert (sorted(best_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_hand passes'

print(test_best_hand())

test_best_hand passes


In [50]:
## problem 2: Jokers
## Deck adds two cards:
## '?B': black joker; can be used as any black card (S or C)
## '?R': red joker; can be used as any red card (H or D)

allranks = '23456789TJQKA'
redcards = [r+s for r in allranks for s in 'DH']
blackcards = [r+s for r in allranks for s in 'SC']

def best_wild_hand(hand):
    "Try all values for jokers in all 5-card selections."
    hands = set(best_hand(h)
               for h in itertools.product(*map(replacements, hand)))
    return max(hands, key = hand_rank)

def replacements(card):
    """Return a list of possible replacements for card.
    there will be more than one only for wild cards."""
    if card == '?B': return blackcards  ## return blackcards means return possible black card, like S/C
    elif card == '?R': return redcards
    else: return [card]

def test_best_wild_hand():
    assert (sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split()))
            == ['7C', '8C', '9C', 'JC', 'TC'])
    assert (sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split()))
            == ['7C', 'TC', 'TD', 'TH', 'TS'])
    ## upper assert need make sure no duplicate cards exist, so
    ## use adding set in hand_rank(modify previous code)
    assert (sorted(best_wild_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_wild_hand passes'

print(test_best_wild_hand())

test_best_wild_hand passes


In [40]:
sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split()))

['7C', 'TC', 'TC', 'TD', 'TD']

List comprehensions provide a concise way to create list. 

In [15]:
squares = []
for x in range(10):
    squares.append(x**2)
    
print(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [16]:
squares2 = list(map(lambda x: x**2, range(10)))
print(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [17]:
[x ** 2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [26]:
matrix = [[1, 2, 3, 4], 
          [5, 6, 7, 8], 
          [9, 10, 11, 12]]

In [27]:
# tranpose rows and columns
[[row[i] for row in matrix] for i in range(4)]

[[1, 5, 9], [2, 6, 10], [3, 7, 11], [4, 8, 12]]

In [28]:
list(zip(*matrix))

[(1, 5, 9), (2, 6, 10), (3, 7, 11), (4, 8, 12)]

In [ ]:
## unpacking Argument list

In [29]:
list(range(3, 6))

[3, 4, 5]

In [33]:
args = [3, 6]
list(range(*args))

[3, 4, 5]

In [35]:
## *-operator to unpack the arguments out of a list or tuple
## dictionaries can deliver keyword arguments with the **-operator: